In [4]:
# !kaggle competitions download -c nfl-big-data-bowl-2024

In [5]:
# !unzip nfl-big-data-bowl-2024.zip

In [9]:
# NOTE: We don't care about the possession team, we have the defensive team as that will be the way we should play our offense.

In [71]:
import numpy as np
import pandas as pd
import scipy.io as sio

# Preprocessing:
Features:
player data:
NFLId, height, weight, birth date, position, age = birth date - game date

play data:
gameID, ballCarrierName, possessionTeam, defensiveTeam, yardstogo, gameClock, quarter, preSnapHomeScore, preSnapVisitorScore, pointsToWin = preSnapHome - preSnapVisitor, absoluteYardlineNumber, offenseFormation, defendersInTheBox, down

game data:
gameID, game date,

Labels:
play data:
playResult

LOG:<br>
Here we vectorize a few fields and then also find Nan values in others.

In [61]:
df = pd.read_csv('processed_dataset.csv')
df

,Unnamed: 0,gameDate,possessionTeam,defensiveTeam,defensiveTeamQt,ballCarrierDisplayName,heightInCm,weightInKg,position,ballCarrierAge,...,preSnapHomeScore,preSnapVisitorScore,signedScoreDiff,absoluteYardlineNumber,offenseFormation,offenseFormationQt,defendersInTheBox,prePenaltyPlayResult,playNullifiedByPenalty,playResult
0,0,10/09/2022,ATL,TB,30,Parker Hesse,190.5,118.39,TE,27.0,...,21,7,-14,69,SHOTGUN,5,7.0,9,N,9
1,1,09/11/2022,PIT,CIN,7,Chase Claypool,193.0,102.97,WR,24.0,...,14,20,6,76,SHOTGUN,5,7.0,3,N,3
2,2,09/11/2022,LV,LAC,18,Darren Waller,198.1,115.67,TE,29.0,...,10,3,-7,40,I_FORM,2,6.0,15,N,15
3,3,10/02/2022,DEN,LV,19,Mike Boone,177.8,92.99,RB,27.0,...,19,16,-3,47,SINGLEBACK,6,6.0,7,N,7
4,4,09/19/2022,BUF,TEN,31,Devin Singletary,170.2,92.08,RB,25.0,...,7,7,0,75,I_FORM,2,7.0,3,N,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12481,12481,10/02/2022,DAL,WAS,32,Ezekiel Elliott,182.9,103.42,RB,27.0,...,0,0,0,49,SINGLEBACK,6,6.0,1,N,1
12482,12482,09/12/2022,SEA,DEN,10,Will Dissly,193.0,121.11,TE,26.0,...,17,16,1,40,SINGLEBACK,6,7.0,2,N,2
12483,12483,10/16/2022,CIN,NO,23,Joe Mixon,185.4,99.79,RB,26.0,...,26,21,-5,69,SHOTGUN,5,6.0,6,N,6
12484,12484,10/02/2022,IND,TEN,31,Jonathan Taylor,180.3,100.24,RB,23.0,...,17,24,-7,44,SHOTGUN,5,6.0,-2,N,-2


In [62]:
# take out the qualitative fields that we aren't using
data = df[["quarter", "down", 'yardsToGo', 'defensiveTeamQt', 'gameClockInSeconds', 'preSnapHomeScore', 'preSnapVisitorScore', 'absoluteYardlineNumber', 'offenseFormationQt', 'defendersInTheBox', 'heightInCm', 'weightInKg', 'position', 'ballCarrierAge', 'playResult']]

In [63]:
data['position'].unique()

array(['TE', 'WR', 'RB', 'QB', 'FB'], dtype=object)

In [64]:
# vectorize 'position' column
data['position'] = data['position'].replace({'TE': 1, 'WR': 2, 'RB': 3, 'QB': 4, 'FB': 5})

/local_scratch/pbs.1584149.pbs02/ipykernel_2442937/162184805.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['position'] = data['position'].replace({'TE': 1, 'WR': 2, 'RB': 3, 'QB': 4, 'FB': 5})


In [65]:
mean_bCA = data['ballCarrierAge'].mean()

# Check for NaN values in the entire DataFrame
data['ballCarrierAge'].fillna(mean_bCA, inplace=True)

/local_scratch/pbs.1584149.pbs02/ipykernel_2442937/3277630479.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['ballCarrierAge'].fillna(mean_bCA, inplace=True)


In [66]:
# Define a function to apply normalization and rounding
normalize_columns = ['defensiveTeamQt', 'offenseFormationQt', 'defendersInTheBox', 'position']

def normalize_and_round(x):
    if x.name not in normalize_columns:  # List the columns to be normalized and rounded
        return ((x - x.mean()) / x.std()).round(2)
    else:
        return x  # Return other columns unchanged

# Apply the custom function to the DataFrame
normalized_data = data.apply(normalize_and_round)

In [67]:
nan_vals = normalized_data.isna()
inf_vals = normalized_data.isin([np.inf, -np.inf])
nan_or_inf = nan_vals | inf_vals
result = nan_or_inf.stack()[nan_or_inf.stack()]
result

1975   defendersInTheBox    True
5770   defendersInTheBox    True
8558   defendersInTheBox    True
9079   defendersInTheBox    True
11460  defendersInTheBox    True
dtype: bool

In [68]:
rows_to_remove = [1975, 5770, 8558, 9079, 11460]
normalized_data.drop(rows_to_remove, axis=0, inplace=True)

In [69]:
nan_vals = normalized_data.isna()
inf_vals = normalized_data.isin([np.inf, -np.inf])
nan_or_inf = nan_vals | inf_vals
result = nan_or_inf.stack()[nan_or_inf.stack()]
result

Series([], dtype: bool)

In [70]:
# # Process the csv file that Rahul created by turning it into .dat file for easy matlab ops
# # Load the CSV file into a DataFrame
# df = pd.read_csv('processed_dataset.csv')

# # Save the DataFrame to a .data file
# df.to_csv('processed_dataset.data', index=False)


# save dataset
# normalized_data.to_csv('processed_dataset.data', index=False) # did not work


output_file = 'processed_dataset.mat'
data_dict = {'normalized_data': normalized_data.to_numpy()}
sio.savemat(output_file, data_dict)